In [1]:
# Reload modules automatically
# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
import importlib
from src.graph_models import create_graph_model
import numpy as np
import torch
import exp_eval_robustness
import exp_train
from torch_sparse import SparseTensor

In [12]:
data_params = dict(
    graph_model = 'CSBM',
    classes = 2,
    n = 1000,
    n_per_class_trn = 400,
    K = 1,
    sigma = 1,
    avg_within_class_degree = 1.58 * 2,
    avg_between_class_degree = 0.37 * 2,
    inductive_samples = 100,
    m=2,
)

model_params = dict(
    label="GATv2",
    model="GATv2",
    #n_filters=64,
    dropout=0.,
    #GATP
    dropout_neighbourhood=0.3,
    n_heads=8,
    n_features_per_head=8,
    #MLP
    hidden_channels=256,
    #APPNP
    #n_hidden=64,
    #K=10,
    #alpha=0.05,
    #LP
    use_label_propagation=False,
    lp_layers=50,
    lp_alpha=0.7,
    lp_use_clamping=False,
)

train_params = dict(
    lr=0.1,
    weight_decay=1e-3,
    patience=300,
    max_epochs=3000,
    inductive=True,
)

attack_params = dict(
    attack = "GRBCD",
    max_robustness = 100,
    surrogate_model_params = dict(
        label="LinearGCN",
        model="LinearGCN",
        n_filter=64,
        dropout=0.5,
    ),
    surrogate_train_params = dict(
        lr=0.1,
        weight_decay=1e-4,
        patience=300,
        max_epochs=3000,
    ),
    power_law_test=True,
)

verbosity_params = dict(
    display_steps = 100,
    debug_lvl = "info"
)  

other_params = dict(
    device = "cpu",
    allow_tf32 = False,
    sacred_metrics = True
)

seed = 8

In [13]:
importlib.reload(exp_eval_robustness)
importlib.reload(exp_train)
#result = exp_train.run(data_params, model_params, train_params, verbosity_params, other_params, seed, None)
result = exp_eval_robustness.run(data_params, model_params, train_params, attack_params, verbosity_params, other_params, seed, None)

2022-11-11 10:05:39 (INFO): Starting experiment exp_eval_robustness with configuration:
2022-11-11 10:05:39 (INFO): data_params: {'graph_model': 'CSBM', 'classes': 2, 'n': 1000, 'n_per_class_trn': 400, 'K': 1, 'sigma': 1, 'avg_within_class_degree': 3.16, 'avg_between_class_degree': 0.74, 'inductive_samples': 100, 'm': 2}
2022-11-11 10:05:39 (INFO): model_params: {'label': 'GATv2', 'model': 'GATv2', 'dropout': 0.0, 'dropout_neighbourhood': 0.3, 'n_heads': 8, 'n_features_per_head': 8, 'hidden_channels': 256, 'use_label_propagation': False, 'lp_layers': 50, 'lp_alpha': 0.7, 'lp_use_clamping': False}
2022-11-11 10:05:39 (INFO): train_params: {'lr': 0.1, 'weight_decay': 0.001, 'patience': 300, 'max_epochs': 3000, 'inductive': True}
2022-11-11 10:05:39 (INFO): attack_params: {'attack': 'GRBCD', 'max_robustness': 100, 'surrogate_model_params': {'label': 'LinearGCN', 'model': 'LinearGCN', 'n_filter': 64, 'dropout': 0.5}, 'surrogate_train_params': {'lr': 0.1, 'weight_decay': 0.0001, 'patience':

KeyboardInterrupt: 

In [4]:
from src.data import split
from src.models import create_model
from src.train import train_inductive, train_transductive

model_params = dict(
    label="GCN",
    model="LinearGCN",
    n_filter=64,
    dropout=0.5,
    use_label_propagation=False,
)
train_params = dict(
    lr=0.1,
    weight_decay=1e-3,
    patience=300,
    max_epochs=3000,
    inductive=True,
)
_run = None
device = torch.device(f"cuda:{0}")

# Sample Graph
graph_model = create_graph_model(data_params)
X_np, A_np, y_np = graph_model.sample(data_params["n"], seed)
X = torch.tensor(X_np, dtype=torch.float32, device=device)
A = torch.tensor(A_np, dtype=torch.float32, device=device)
y = torch.tensor(y_np, device=device)
split_trn, split_val = split(y_np, data_params)

# Create Model
model_params_trn = dict(**model_params, 
                        n_features=X_np.shape[1], 
                        n_classes=data_params["classes"])
model = create_model(model_params_trn).to(device)
#logging.info(model)

# Train Model
if train_params["inductive"]:
    train = train_inductive
else:
    train = train_transductive
trn_tracker = train(model, None, X, A, y, split_trn, split_val, train_params,
                    verbosity_params, _run)

2022-11-08 16:00:42 (INFO): 
Epoch    0: loss_train: 0.74230, loss_val: 0.74731, acc_train: 0.32375, acc_val: 0.23500
2022-11-08 16:00:42 (INFO): 
Epoch  100: loss_train: 0.21736, loss_val: 0.27384, acc_train: 0.91625, acc_val: 0.89500
2022-11-08 16:00:43 (INFO): 
Epoch  200: loss_train: 0.21414, loss_val: 0.27090, acc_train: 0.92000, acc_val: 0.89500
2022-11-08 16:00:43 (INFO): 
Epoch  300: loss_train: 0.20504, loss_val: 0.27502, acc_train: 0.91625, acc_val: 0.89000
2022-11-08 16:00:44 (INFO): 
Epoch  400: loss_train: 0.21400, loss_val: 0.26903, acc_train: 0.91125, acc_val: 0.90500
2022-11-08 16:00:44 (INFO): 
Epoch  500: loss_train: 0.21238, loss_val: 0.27094, acc_train: 0.91125, acc_val: 0.90000
2022-11-08 16:00:44 (INFO): 
Epoch  600: loss_train: 0.21301, loss_val: 0.26617, acc_train: 0.92375, acc_val: 0.89500
2022-11-08 16:00:45 (INFO): 
Epoch  366: loss_train: 0.22549, loss_val: 0.24665, acc_train: 0.90500, acc_val: 0.90500


In [14]:
W1 = model.layers[0][0]._linear.weight.T
W2 = model.layers[1][0]._linear.weight.T
print(W1.shape)
print(W2.shape)
print(W1.matmul(W2).shape)

torch.Size([21, 64])
torch.Size([64, 2])
torch.Size([21, 2])


In [15]:
print(model)

DenseGCN(
  (activation): Identity()
  (layers): ModuleList(
    (0): Sequential(
      (gcn_0): DenseGraphConvolution(
        (_linear): Linear(in_features=21, out_features=64, bias=False)
      )
      (activation_0): Identity()
      (dropout_0): Dropout(p=0.5, inplace=False)
    )
    (1): Sequential(
      (gcn_1): DenseGraphConvolution(
        (_linear): Linear(in_features=64, out_features=2, bias=False)
      )
      (softmax_1): LogSoftmax(dim=1)
    )
  )
)


In [18]:
from src.models.gcn import DenseGCN
isinstance(model, DenseGCN)

True

In [22]:
isinstance(model.activation, torch.nn.Identity)

True

In [25]:
model.layers[0][0]._linear.weight

Parameter containing:
tensor([[-0.2845, -0.1751,  0.0541,  ...,  0.3195,  0.0939,  0.0529],
        [-0.0133, -0.1708,  0.0041,  ..., -0.0677, -0.3852, -0.0863],
        [ 0.0237, -0.1400,  0.0761,  ..., -0.0926, -0.1117, -0.0352],
        ...,
        [-0.1547, -0.6180,  0.0105,  ..., -0.4443, -0.1026, -0.1262],
        [-0.0155,  0.1586,  0.4082,  ...,  0.2353, -0.1754, -0.0009],
        [-0.1066, -0.2037, -0.0348,  ..., -0.1079, -0.1091, -0.2001]],
       device='cuda:0', requires_grad=True)

In [26]:
model_params = dict(
    label="SGC",
    model="SGC",
    K=2,
    use_label_propagation=False,
)
model_params_trn = dict(**model_params, 
                        n_features=X_np.shape[1], 
                        n_classes=data_params["classes"])
sgc = create_model(model_params_trn).to(device)

In [45]:
model.layers[0][0]

DenseGraphConvolution(
  (_linear): Linear(in_features=21, out_features=64, bias=False)
)

In [39]:
sgc.parameters

<bound method Module.parameters of SGC(
  (sgc): SGConv(21, 2, K=2)
)>

In [16]:
def gen():
    for i in range(10):
        yield i
    yield None


mygen = gen()
def call_gen():
    return next(mygen)

for i in range(11):
    print(call_gen())

0
1
2
3
4
5
6
7
8
9
None


In [15]:
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"]

KeyError: 'CUBLAS_WORKSPACE_CONFIG'

In [9]:
data_params["sigma"] = 1
seed = 1

result = exp_eval_robustness.run(data_params, model_params, train_params, attack_params, verbosity_params, other_params, seed, None)

2022-08-10 14:11:26 (INFO): Starting experiment exp_eval_robustness with configuration:
2022-08-10 14:11:26 (INFO): data_params: {'graph_model': 'CSBM', 'classes': 2, 'n': 1000, 'n_per_class_trn': 400, 'K': 0.5, 'sigma': 1, 'avg_within_class_degree': 3.0, 'avg_between_class_degree': 1.0, 'inductive_samples': 1000}
2022-08-10 14:11:26 (INFO): model_params: {'label': 'GCN', 'model': 'DenseGCN', 'n_filters': 64}
2022-08-10 14:11:26 (INFO): train_params: {'lr': 0.01, 'weight_decay': 0.001, 'patience': 300, 'max_epochs': 3000, 'inductive': True}
2022-08-10 14:11:26 (INFO): attack_params: {'attack': 'l2'}
2022-08-10 14:11:26 (INFO): verbosity_params: {'display_steps': 100, 'debug_lvl': 'info'}
2022-08-10 14:11:26 (INFO): other_params: {'device': 0, 'allow_tf32': False, 'sacred_metrics': True}
2022-08-10 14:11:26 (INFO): seed: 1
2022-08-10 14:11:26 (INFO): db_collection: None
2022-08-10 14:11:26 (INFO): Currently on gpu device cuda:0
2022-08-10 14:11:27 (INFO): 
Epoch    0: loss_train: 0.7010

In [21]:
from src.data import calc_balanced_sample

In [22]:
class_counts = [10, 100, 100]
n_samples = 60
print(calc_balanced_sample(class_counts, n_samples))

[10 25 25]


In [35]:
from src.utils import accuracy

In [7]:
logits = torch.Tensor([[0.8, 0.2], [0.1, 0.9]])
labels = torch.Tensor([0, 1])
logits.argmax(1)[1]

tensor(1)